# Document an Excel-based application scorecard model

Use ValidMind to build and document an Excel-based *application scorecard model*.


An application scorecard model is a type of statistical model used in credit scoring to evaluate the creditworthiness of potential borrowers by generating a score based on various characteristics of an applicant — such as credit history, income, employment status, and other relevant financial data. 

- This score helps lenders make decisions about whether to approve or reject loan applications, as well as determine the terms of the loan, including interest rates and credit limits. 
- Application scorecard models enable lenders to manage risk efficiently while making the loan application process faster and more transparent for applicants.

Learn how to load an Excel-based model, assign predictions for testing, set up test inputs, run tests, and then log the test results to ValidMind.


<a id='toc1_'></a>

## About ValidMind
ValidMind is a suite of tools for managing model risk, including risk associated with AI and statistical models.

You use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin
This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?
If you haven't already seen our [Get started with the ValidMind Library](https://docs.validmind.ai/developer/get-started-validmind-library.html), we recommend you begin by exploring the available resources in this section. There, you can learn more about documenting models, find code samples, or read our developer reference.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, you'll need access to a ValidMind account.</b></span>
<br></br>
<a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>


<a id='toc1_3_'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Library, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Custom tests**: Custom tests are functions that you define to evaluate your model or dataset. These functions can be registered via the ValidMind Library to be used with the ValidMind Platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind Library. They can be any of the following:

- **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
- **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
- **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom test.
- **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom test. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a test, customize its behavior, or provide additional context.

**Outputs**: Custom tests can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: The [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.

<a id='toc2_'></a>

## Install the ValidMind Library

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2); border-radius: 5px;">
  <span style="color: #083E44;"><b>Recommended Python versions</b></span><br />
  Python 3.8 ≤ x ≤ 3.11
</div>


To install the library:

In [ ]:
%pip install -q validmind

<a id='toc3_'></a>

## Initialize the ValidMind Library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

<a id='toc3_1_'></a>

### Get your code snippet

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register Model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Credit Risk Scorecard`
   - Use case: `Credit Risk - CECL`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, [load your model identifier credentials from an `.env` file](https://docs.validmind.ai/developer/model-documentation/store-credentials-in-env-file.html) or replace the placeholder with your own code snippet:

In [ ]:
# Load your model identifier credentials from an `.env` file

%load_ext dotenv
%dotenv .env

# Or replace with your code snippet

import validmind as vm

vm.init(
    # api_host="...",
    # api_key="...",
    # api_secret="...",
    # model="...",
)

<a id='toc4_'></a>

## Initialize the Python environment

Since we are working with excel files, we need to install the `openpyxl` library:

In [ ]:
%pip install openpyxl


Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from validmind.datasets.credit_risk import lending_club
import pandas as pd
import openpyxl
import xgboost as xgb

%matplotlib inline

<a id='toc4_1_'></a>

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You'll upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

<a id='toc5_'></a>

## Load the sample dataset

You'll need to [download](/CreditRiskData.xlsx) the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

The dataset is provided as an Excel workbook with five sheets, each representing a different stage of data preparation. These sheets are indexed from 0 to 4:

- **Raw Data** – The original unprocessed dataset.
- **Pre-Process Data** – A cleaned and prepared version of the raw data.
- **FE Data** – Feature-engineered data with new or transformed variables.
- **Train Data** – A training subset used to fit your model.
- **Test Data** – A testing subset used to evaluate model performance.

We will start by loading the **Raw Data** sheet (index `0`) into a pandas DataFrame:

In [10]:
df = pd.read_excel('CreditRiskData.xlsx', sheet_name=0,engine='openpyxl')

df.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,open_acc,total_acc,total_pymnt
0,5000,4975.0,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,0,credit_card,3,9,5861.071414
1,2500,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,1,car,3,4,1008.710000
2,2400,2400.0,36 months,15.96,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,0,small_business,2,10,3003.653644
3,10000,10000.0,36 months,13.49,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,0,other,10,37,12226.302210
4,5000,5000.0,36 months,7.90,156.46,A,A4,3 years,RENT,36000.0,Source Verified,0,wedding,9,12,5631.377753


<a id='toc5_1_'></a>

### Prepocess the dataset

In the preprocessing step we perform a number of operations to get ready for building our application scorecard. A few of those steps include: 

- Filtering the dataset to focus on relevant loan types.
- Removing riskier loans.
- Dropping irrelevant fields and entries with missing data.

We’ll load the **Pre-Process Data** sheet (index `1`) into a pandas DataFrame:

In [11]:
preprocess_df = pd.read_excel('CreditRiskData.xlsx', sheet_name=1,engine='openpyxl')
preprocess_df.head()

,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,open_acc,total_acc
0,36,10.65,162.87,B,B2,10,RENT,24000.0,Verified,0,credit_card,3,9
1,60,14.65,153.45,C,C3,5,OWN,72000.0,Not Verified,0,debt_consolidation,14,23
2,36,12.69,402.54,B,B5,10,OWN,75000.0,Source Verified,0,debt_consolidation,12,34
3,36,13.49,305.38,C,C1,0,RENT,30000.0,Source Verified,1,debt_consolidation,4,9
4,36,9.91,96.68,B,B1,3,RENT,15000.0,Source Verified,0,credit_card,11,11


<a id='toc5_2_'></a>

### Feature engineering

In the feature engineering phase, we apply specific transformations to optimize the dataset for predictive modeling in our application scorecard. 

- **WoE encoding**: Converts both numerical and categorical features into Weight of Evidence (WoE) values. WoE is a statistical measure that captures the relationship between a predictor variable and the binary target variable. This transformation helps make features more predictive and consistent in their contribution to the model.

- **Integration of WoE bins**: Ensures that the WoE transformed values are integrated throughout the dataset, replacing the original feature values while excluding the target variable from this transformation. Integrating the WoE values helps maintain consistent scaling across inputs and improves overall model stability.

We’ll now load the feature-engineered dataset from the **FE Data sheet** (index `2`) into a pandas DataFrame:

In [12]:
fe_df = pd.read_excel('CreditRiskData.xlsx', sheet_name=2,engine='openpyxl')
fe_df.head()

,loan_status,installment_woe,term_woe,annual_inc_woe,sub_grade_woe,verification_status_woe,purpose_woe,grade_woe,home_ownership_woe,total_acc_woe,int_rate_woe,open_acc_woe,emp_length_woe
0,0,-0.191738,-0.215312,0.272855,-0.336648,0.144895,-0.157770,-0.396915,0.114461,0.085175,-0.202397,-0.135784,-0.016348
1,0,-0.191738,0.654502,-0.054231,0.337916,-0.243789,0.051351,0.186188,0.053303,0.032958,-0.202397,0.023460,-0.016348
2,0,0.128420,-0.215312,-0.054231,-0.336648,0.089350,0.051351,-0.396915,0.053303,-0.101325,-0.202397,0.023460,-0.016348
3,1,-0.013370,-0.215312,0.272855,-0.336648,0.089350,0.051351,0.186188,0.114461,0.085175,-0.202397,-0.135784,0.053758
4,0,-0.191738,-0.215312,0.272855,-0.336648,0.089350,-0.157770,-0.396915,0.114461,0.085175,-1.067745,0.023460,-0.016348


<a id='toc6_'></a>

## Bring In Model and Assign Prediction

In this step, we prepare the model using feature-engineered data already split into training (**Train Data**, index `3`) and testing (**Test Data**, index `4`) sets. We'll load those datasets into pandas DataFrames:


In [13]:
# Split the data
train_df = pd.read_excel('CreditRiskData.xlsx', sheet_name=3,engine='openpyxl') 
test_df = pd.read_excel('CreditRiskData.xlsx', sheet_name=4,engine='openpyxl')

<a id='toc7_'></a>

## Document the model

To document the model with the ValidMind Library, you'll need to:
1. Preprocess the raw dataset
2. Initialize some training and test datasets
3. Initialize a model object you can use for testing
4. Run the full suite of tests

<a id='toc7_1_'></a>

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize each dataset using [`init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset`: The input DataFrame to test.
- `input_id`: A unique identifier for tracking test inputs.
- `target_column`: Required for tests that compare predictions to actual outcomes; specify the name of the column with the true values.

Initialize each of the datasets created earlier (`raw_df`, `preprocessed_df`, `fe_df`, `train_df`, `test_df`) as ValidMind dataset objects:


In [14]:
vm_raw_dataset = vm.init_dataset(
    dataset=df,
    input_id="raw_dataset",
    target_column='loan_status',
)

vm_preprocess_dataset = vm.init_dataset(
    dataset=preprocess_df,
    input_id="preprocess_dataset",
    target_column='loan_status',
)

vm_fe_dataset = vm.init_dataset(
    dataset=fe_df,
    input_id="fe_dataset",
    target_column='loan_status',
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column='loan_status',
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    input_id="test_dataset",
    target_column='loan_status',
)

<a id='toc7_3_'></a>

### Initialize a model object

In this Excel-based use case, predictions are precomputed and included in the Excel file. While there's no model logic to run, ValidMind still requires a model object to link predictions to the documentation workflow.

To satisfy this requirement, define a placeholder model using [`init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):


In [15]:
#need to assign a model, this is just a dummy function
def dummy():
    return None

xgb_model = vm.init_model(
        #xgb_model,
        input_id="xgb_model",
        predict_fn=dummy
    )

### Assign prediction values and probabilities to the datasets

Since predictions already exist in the Excel file, use the `assign_predictions()` method on the `vm` dataset objects to link prediction values and probabilities to the train and test datasets:


In [16]:
# Assign Predictions from your dataset 
vm_train_ds.assign_predictions(model=xgb_model, prediction_column="xgb_model_prediction",probability_column='xgb_model_probabilities')
vm_test_ds.assign_predictions(model=xgb_model, prediction_column="xgb_model_prediction",probability_column='xgb_model_probabilities')

### Add custom context to LLM-generated test descriptions

To generate proper summaries for test outputs, you can enable LLM descriptions with custom formatting.

Set the `VALIDMIND_LLM_DESCRIPTIONS_CONTEXT_ENABLED` environment variable to `1` to enable the feature. Then define the `VALIDMIND_LLM_DESCRIPTIONS_CONTEXT` variable with a structured prompt to control how the LLM explains test results.

This configuration applies globally and ensures that all generated test descriptions follow your specified format and content structure.


In [17]:
import os
os.environ["VALIDMIND_LLM_DESCRIPTIONS_CONTEXT_ENABLED"] = "1"

context = """
FORMAT FOR THE LLM DESCRIPTIONS: 
    **<Test Name>** is designed to <begin with a concise overview of what the test does and its primary purpose, 
    extracted from the test description>.

    The test operates by <write a paragraph about the test mechanism, explaining how it works and what it measures. 
    Include any relevant formulas or methodologies mentioned in the test description.>

    The primary advantages of this test include <write a paragraph about the test's strengths and capabilities, 
    highlighting what makes it particularly useful for specific scenarios.>

    Users should be aware that <write a paragraph about the test's limitations and potential risks. 
    Include both technical limitations and interpretation challenges. 
    If the test description includes specific signs of high risk, incorporate these here.>

    **Key Insights:**

    The test results reveal:

    - **<insight title>**: <comprehensive description of one aspect of the results>
    - **<insight title>**: <comprehensive description of another aspect>
    ...

    Based on these results, <conclude with a brief paragraph that ties together the test results with the test's 
    purpose and provides any final recommendations or considerations.>

ADDITIONAL INSTRUCTIONS:
    Present insights in order from general to specific, with each insight as a single bullet point with bold title.

    For each metric in the test results, include in the test overview:
    - The metric's purpose and what it measures
    - Its mathematical formula
    - The range of possible values
    - What constitutes good/bad performance
    - How to interpret different values

    Each insight should progressively cover:
    1. Overall scope and distribution
    2. Complete breakdown of all elements with specific values
    3. Natural groupings and patterns
    4. Comparative analysis between datasets/categories
    5. Stability and variations
    6. Notable relationships or dependencies

    Remember:
    - Keep all insights at the same level (no sub-bullets or nested structures)
    - Make each insight complete and self-contained
    - Include specific numerical values and ranges
    - Cover all elements in the results comprehensively
    - Maintain clear, concise language
    - Use only "- **Title**: Description" format for insights
    - Progress naturally from general to specific observations

""".strip()

os.environ["VALIDMIND_LLM_DESCRIPTIONS_CONTEXT"] = context

<a id='toc6_5_'></a>

### Run the full suite of tests

This is where it all comes together, you are now ready to run the documentation tests for the model as defined by the documentation template you looked at earlier.

We will first define a function (`Run_Doc_Test`) to run a set of specified test:

In [18]:
import validmind as vm
def Run_Doc_Test(test_config):
    for t in test_config:
        print(t)
        try:
            # Check if test has input_grid
            if 'input_grid' in test_config[t]:
                # For tests with input_grid, pass the input_grid configuration
                if 'params' in test_config[t]:
                    vm.tests.run_test(t, input_grid=test_config[t]['input_grid'], params=test_config[t]['params'],show=False).log()
                else:
                    vm.tests.run_test(t, input_grid=test_config[t]['input_grid'],show=False).log()
            else:
                # Original logic for regular inputs
                if 'params' in test_config[t]:
                    vm.tests.run_test(t, inputs=test_config[t]['inputs'], params=test_config[t]['params'],show=False).log()
                else:
                    vm.tests.run_test(t, inputs=test_config[t]['inputs'],show=False).log()
        except Exception as e:
            print(f"Error running test {t}: {str(e)}")


The function requires information about the inputs to use on every test. These inputs can be passed as an `inputs` argument if we want to use the same inputs for all tests. It's also possible to pass a `config` argument that has information about the `params` and `inputs` that each test requires. The `config` parameter is a dictionary with the following structure:

```python
config = {
    "<test-id>": {
        "params": {
            "param1": "value1",
            "param2": "value2",
            ...
        },
        "inputs": {
            "input1": "value1",
            "input2": "value2",
            ...
        }
    },
    ...
}
```

We will define set of tests in `test_config2` following the structure of the `config` parameter. Each `<test-id>` corresponds to the test driven block identifiers shown by `vm.preview_template()`:

In [19]:
#Explicit Test Configuration 
test_config2 = {'validmind.data_validation.DatasetDescription:raw_data': {'inputs': {'dataset': 'raw_dataset'}},
 'validmind.data_validation.DescriptiveStatistics:raw_data': {'inputs': {'dataset': 'raw_dataset'}},
 'validmind.data_validation.MissingValues:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'min_threshold': 1}},
 'validmind.data_validation.ClassImbalance:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'min_percent_threshold': 10}},
 'validmind.data_validation.Duplicates:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'min_threshold': 1}},
 'validmind.data_validation.HighCardinality:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'num_threshold': 100,
   'percent_threshold': 0.1,
   'threshold_type': 'percent'}},
 'validmind.data_validation.Skewness:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'max_threshold': 1}},
 'validmind.data_validation.UniqueRows:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'min_percent_threshold': 1}},
 'validmind.data_validation.TooManyZeroValues:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'max_percent_threshold': 0.03}},
 'validmind.data_validation.IQROutliersTable:raw_data': {'inputs': {'dataset': 'raw_dataset'},
  'params': {'threshold': 5}},
 'validmind.data_validation.DescriptiveStatistics:preprocessed_data': {'inputs': {'dataset': 'preprocess_dataset'}},
 'validmind.data_validation.TabularDescriptionTables:preprocessed_data': {'inputs': {'dataset': 'preprocess_dataset'}},
 'validmind.data_validation.MissingValues:preprocessed_data': {'inputs': {'dataset': 'preprocess_dataset'},
  'params': {'min_threshold': 1}},
 'validmind.data_validation.TabularNumericalHistograms:preprocessed_data': {'inputs': {'dataset': 'preprocess_dataset'}},
 'validmind.data_validation.TabularCategoricalBarPlots:preprocessed_data': {'inputs': {'dataset': 'preprocess_dataset'}},
 'validmind.data_validation.TargetRateBarPlots:preprocessed_data': {'inputs': {'dataset': 'preprocess_dataset'},
  'params': {'default_column': 'loan_status'}},
 'validmind.data_validation.DescriptiveStatistics:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']}},
 'validmind.data_validation.TabularDescriptionTables:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']}},
 'validmind.data_validation.ClassImbalance:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']},
  'params': {'min_percent_threshold': 10}},
 'validmind.data_validation.UniqueRows:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']},
  'params': {'min_percent_threshold': 1}},
 'validmind.data_validation.TabularNumericalHistograms:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']}},
 'validmind.data_validation.MutualInformation:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']},
  'params': {'min_threshold': 0.01}},
 'validmind.data_validation.PearsonCorrelationMatrix:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']}},
 'validmind.data_validation.HighPearsonCorrelation:development_data': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']},
  'params': {'max_threshold': 0.3, 'top_n_correlations': 10}},
 'validmind.data_validation.WOEBinTable': {'input_grid': {'dataset': ['preprocess_dataset']},
  'params': {'breaks_adj': {'loan_amnt': [5000, 10000, 15000, 20000, 25000],
    'int_rate': [10, 15, 20],
    'annual_inc': [50000, 100000, 150000]}}},
 'validmind.data_validation.WOEBinPlots': {'input_grid': {'dataset': ['preprocess_dataset']},
  'params': {'breaks_adj': {'loan_amnt': [5000, 10000, 15000, 20000, 25000],
    'int_rate': [10, 15, 20],
    'annual_inc': [50000, 100000, 150000]}}},
 'validmind.data_validation.DatasetSplit': {'inputs': {'datasets': ['train_dataset',
    'test_dataset']}},
 'validmind.model_validation.statsmodels.GINITable': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']}},
 'validmind.model_validation.sklearn.ClassifierPerformance': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']}},
 'validmind.model_validation.sklearn.TrainingTestDegradation:XGBoost': {'inputs': {'datasets': ['train_dataset',
    'test_dataset'],
   'model': 'xgb_model'},
  'params': {'max_threshold': 0.1}},
 'validmind.model_validation.sklearn.ROCCurve': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']}},
 'validmind.model_validation.sklearn.MinimumROCAUCScore': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']},
  'params': {'min_threshold': 0.5}},
 'validmind.model_validation.statsmodels.PredictionProbabilitiesHistogram': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']}},
 'validmind.model_validation.statsmodels.CumulativePredictionProbabilities': {'input_grid': {'model': ['xgb_model'],
   'dataset': ['train_dataset', 'test_dataset']}},
 'validmind.model_validation.sklearn.PopulationStabilityIndex': {'inputs': {'datasets': ['train_dataset',
    'test_dataset'],
   'model': 'xgb_model'},
  'params': {'num_bins': 10, 'mode': 'fixed'}},
 'validmind.model_validation.sklearn.ConfusionMatrix': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']}},
 'validmind.model_validation.sklearn.MinimumAccuracy': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']},
  'params': {'min_threshold': 0.7}},
 'validmind.model_validation.sklearn.MinimumF1Score': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']},
  'params': {'min_threshold': 0.5}},
 'validmind.model_validation.sklearn.PrecisionRecallCurve': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']}},
 'validmind.model_validation.sklearn.CalibrationCurve': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset'],
   'model': ['xgb_model']}},
 'validmind.model_validation.sklearn.ClassifierThresholdOptimization': {'inputs': {'dataset': 'train_dataset',
   'model': 'xgb_model'},
  'params': {'target_recall': 0.8}},
 'validmind.model_validation.statsmodels.ScorecardHistogram': {'input_grid': {'dataset': ['train_dataset',
    'test_dataset']},
  'params': {'score_column': 'xgb_scores'}},
 'validmind.data_validation.ScoreBandDefaultRates': {'input_grid': {'dataset': ['train_dataset'],
   'model': ['xgb_model']},
  'params': {'score_column': 'xgb_scores', 'score_bands': [504, 537, 570]}},
 'validmind.model_validation.sklearn.ScoreProbabilityAlignment': {'input_grid': {'dataset': ['train_dataset'],
   'model': ['xgb_model']},
  'params': {'score_column': 'xgb_scores'}},
 'validmind.model_validation.sklearn.WeakspotsDiagnosis': {'inputs': {'datasets': ['train_dataset',
    'test_dataset'],
   'model': 'xgb_model'}},
 'validmind.model_validation.sklearn.OverfitDiagnosis': {'inputs': {'model': 'xgb_model',
   'datasets': ['train_dataset', 'test_dataset']},
  'params': {'cut_off_threshold': 0.04}},
 'validmind.model_validation.sklearn.RobustnessDiagnosis': {'inputs': {'datasets': ['train_dataset',
    'test_dataset'],
   'model': 'xgb_model'},
  'params': {'scaling_factor_std_dev_list': [0.1, 0.2, 0.3, 0.4, 0.5],
   'performance_decay_threshold': 0.05}},
 'validmind.model_validation.FeaturesAUC': {'input_grid': {'model': ['xgb_model'],
   'dataset': ['train_dataset', 'test_dataset']}},}


Now we can pass the input configuration to `Run_Doc_Test` and run the full suite of tests. The variable `full_suite` then holds the result of these tests:

In [21]:
full_suite = Run_Doc_Test(test_config2)

validmind.model_validation.sklearn.OverfitDiagnosis
validmind.model_validation.sklearn.RobustnessDiagnosis
Error running test validmind.model_validation.sklearn.RobustnessDiagnosis: dummy() takes 0 positional arguments but 1 was given
validmind.model_validation.FeaturesAUC


2025-07-29 12:48:48,488 - INFO(validmind.vm_models.result.result): Test driven block with result_id validmind.model_validation.FeaturesAUC does not exist in model's document
2025-07-29 12:48:48,489 - WARNING(validmind.ai.test_descriptions): Failed to generate description for validmind.model_validation.FeaturesAUC: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

2025-07-29 12:48:48,493 - WARNING(validmind.ai.test_descriptions): Using default description for validmind.model_validation.FeaturesAUC
2025-07-29 12:48:48,495 - ERROR(validmind.api_client): Error logging figure to ValidMind API
2025-07-29 12:48:48,499 - ERROR(validmind.api_client): Error logging figure to ValidMind API


Error running test validmind.model_validation.FeaturesAUC: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



<a id='toc8_'></a>

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way, use the ValidMind Platform to work with your model documentation.

<a id='toc8_1_'></a>

### Work with your model documentation

1. In the ValidMind Platform, go to the **Documentation** page for the model you registered earlier. ([Need more help?](https://docs.validmind.ai/guide/model-documentation/working-with-model-documentation.html))

2. Expand the following sections and take a look around:

   - **2. Data Preparation**
   - **3. Model Development**

What you see is the full draft of your model documentation in a more easily consumable version. From here, you can make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready.

<a id='toc8_2_'></a>

### Discover more learning resources

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/developer/model-testing/testing-overview.html)
- [Code samples](https://docs.validmind.ai/developer/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.

<a id='toc9_'></a>

## Upgrade ValidMind

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;">After installing ValidMind, you’ll want to periodically make sure you are on the latest version to access any new features and other enhancements.</div>

Retrieve the information for the currently installed version of ValidMind:

In [ ]:
%pip show validmind

If the version returned is lower than the version indicated in our [production open-source code](https://github.com/validmind/validmind-library/blob/prod/validmind/__version__.py), restart your notebook and run:

```bash
%pip install --upgrade validmind
```

You may need to restart your kernel after running the upgrade package for changes to be applied.